# Cortes Economicos

El costo de hacer un corte de una madera de longitud `l` es $l$ (cualquiera sea el corte).

Tenemos que cortaruna madera en distintos puntos de corte $C_i \in {0,..,l}$ almacenados en $C$.

Queremos encontrar el minimo costo posible de cortar una vara de longitud $l$ en los puntos de corte determinados $C$.

$$ \text{CostoMin}(i, j) = \begin{cases} 0 & \text{si no hay puntos de corte entre } i \text{ y } j \
\\
   \min_{c \in C, i < c < j} \left( (j-i) + \text{CostoMin}(i, c) + \text{CostoMin}(c, j) \right) & \text{de lo contrario} \end{cases} $$

### Algoritmo de Programacion Dinamica - Top Down

* Complejidad temporal: $O(|C|^3)$, porque se hay $|C|^2$ estados unicos (subproblemas a resolver), con un coste de O($|C|$) cada uno (iteramos sobre todos los cortes posibles entre `i` y `j`).

* Complejidad espacial: $O(|C|^2)$, porque la matriz auxiliar mide $L^2$, ya que el dominio de posibles llamados es de ${0..L}^2$

In [86]:
import numpy as np

def CostoMin(C, i, j, M):
    # intento aplicar memoizacion
    if M[i][j] != -1:
        return M[i][j]

    # obtengo los puntos de corte en el rango [i, j]
    elementosEnRango = list(filter(lambda x: i <= x and x <= j, C))

    # caso base 
    if len(elementosEnRango) == 0:
        return 0

    # caso recursivo   
    minCostInterno = float('inf')
    for elem in elementosEnRango:
        C.remove(elem)
        cost = CostoMin(C, i, elem, M) + CostoMin(C, elem, j, M) + (j-i)
        if cost <= minCostInterno:
            minCostInterno = cost              
        C.append(elem)

    # guardo el resultado en la matriz de memoizacion
    M[i][j] = minCostInterno
    
    return M[i][j]


def solve(C, L):
    # inicializo la matriz de memoizacion
    M = np.empty((L+1,L+1), dtype=int)
    M.fill(-1)

    # ordeno los costos para que el algoritmo sea mas eficiente
    C.sort()                                            
    
    # llamo a la funcion recursiva
    return CostoMin(C, 0, L, M)

In [87]:
solve([2,4,7], 10)

20

### Algoritmo de Programacion Dinamica - Bottom up

- Complejidad Temporal: $O(|C|^3)$
- Complejidad Espacial: $O(|C|^2)$

In [ ]:
# El procedimiento sería: 
# 1. Inicializar la matriz de dp (dynamic programmation) con ceros para los casos base. 
# 2. Llenar la matriz de dp iterativamente, utilizando la fórmula para calcular el
#     mínimo costo de cortar entre cada par de puntos de corte.

def CostMinBottomUp(C, L):
    # inicializo la matriz de memoizacion
    M = np.empty((L+1,L+1), dtype=int)
    M.fill(-1)

    # ordeno los costos para que el algoritmo sea mas eficiente
    C.sort()                                            
    
    # caso base
    for i in range(0, L+1):
        M[i][i] = 0

    # paso por todos los estados posibles para armar la matriz de memoizacion M
    for l in range(2, L+1):
        # Por cada longitud posible L    
        for i in range(0, L-l+1):
            # Por cada punto de corte posible desde `i` a `l` 
            j = i + l
            minCostInterno = float('inf')
            # Aca los posibles C son todos los elementos que estan en el rango [i, j]
            for k in range(i+1, j):
                cost = M[i][k] + M[k][j] + (j-i)
                if cost <= minCostInterno:
                    minCostInterno = cost
            M[i][j] = minCostInterno

    # devuelvo el valor de la solucion pedido
    return M[0][L]

#### Una observacion sobre las complejidades en TopDown y BottomUp:

La razón por la que la complejidad espacial es ($O(|C|^2)$) y no ($O(\ell^2)$) se debe a que la cantidad de subproblemas que necesitamos resolver (y por lo tanto, el tamaño de nuestra tabla de memorización) está directamente relacionada con el número de cortes posibles, es decir, el tamaño de ($C$), más los dos cortes adicionales (el inicio y el final de la vara), no con la longitud física ($\ell$) de la vara.

Para cada par de puntos de corte, necesitamos calcular y almacenar el costo mínimo de realizar los cortes entre esos dos puntos. Dado que estamos interesados en todos los pares posibles de puntos de corte, el número total de entradas en nuestra tabla será proporcional al cuadrado del número de puntos de corte, es decir, ($O((|C|+2)^2)$), lo cual simplificamos como ($O(|C|^2)$) para fines de análisis de complejidad.

El análisis de la longitud física ($\ell$) de la vara no es relevante para la complejidad espacial del algoritmo de PD porque no estamos almacenando resultados intermedios para cada posible longitud de subvara a lo largo de la vara original, sino solo para los segmentos definidos por los puntos de corte específicos en ($C$). Esto es un aspecto crucial de la programación dinámica: optimizar el uso de la memoria al limitar el número de subproblemas que necesitamos resolver, basándonos en las características específicas del problema.

En conclusión, la complejidad espacial es ($O(|C|^2)$) porque el número de subproblemas (segmentos entre cortes) que necesitamos considerar crece con el cuadrado del número de puntos de corte, no con el cuadrado de la longitud total de la vara.


Creo: "Si bien es mide $\ell^2$ la matriz, solo uso $|C|^2$ celdas y podria ahorrarse el resto de memoria con otra estructura".

**Es decir, si bien los `i`, `j` se mueven de 0 a $\ell$, solo lo hacen de a saltos por los $c \in C$.**